In [ ]:
!pip install gradio transformers

In [ ]:
!pip install gradio transformers networkx

In [ ]:
!pip install gTTS pydub

In [ ]:
import gradio as gr
import networkx as nx
import os
from gtts import gTTS
from transformers import pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
# Load a Hugging Face LLM
llm = pipeline("text2text-generation", model="google/flan-t5-large")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Define a dictionary to represent the hospital layout (nodes and edges)
hospital_graph = nx.Graph()

# Add nodes for rooms/departments on each floor based on the floor plans
# Add nodes for 1st floor
hospital_graph.add_node("Hospital_1F_Entrance", floor=1, x=100, y=100, description="Main Entrance")
hospital_graph.add_node("Hospital_1F_Reception", floor=1, x=150, y=100, description="Main Registration and Reception")
hospital_graph.add_node("Hospital_1F_Emergency", floor=1, x=200, y=150, description="Emergency Department")
hospital_graph.add_node("Hospital_1F_Womens_Imaging", floor=1, x=220, y=180, description="Women's Imaging Center")
hospital_graph.add_node("Hospital_1F_Diagnostic_Imaging", floor=1, x=250, y=200, description="Diagnostic Imaging")
hospital_graph.add_node("Hospital_1F_Laboratory", floor=1, x=270, y=200, description="Laboratory")
hospital_graph.add_node("Hospital_1F_Courtyard_Cafe", floor=1, x=300, y=210, description="Courtyard Cafe")
hospital_graph.add_node("Hospital_1F_Conference_Center", floor=1, x=320, y=220, description="Conference Center")
# Add nodes for 2nd floor
hospital_graph.add_node("Hospital_2F_Critical_Care", floor=2, x=100, y=100, description="Critical Care Unit")
hospital_graph.add_node("Hospital_2F_Med_Surg", floor=2, x=150, y=120, description="Med/Surg Unit")
hospital_graph.add_node("Hospital_2F_Surgery", floor=2, x=180, y=130, description="Surgery Department")
hospital_graph.add_node("Hospital_2F_Outpatient_Infusion", floor=2, x=200, y=140, description="Outpatient Infusion")
hospital_graph.add_node("Hospital_2F_Nurse_Station", floor=2, x=220, y=150, description="Nurse Station")
# Add nodes for 3rd floor
hospital_graph.add_node("Hospital_3F_Human_Resources", floor=3, x=100, y=100, description="Human Resources")
hospital_graph.add_node("Hospital_3F_Learning_Center", floor=3, x=150, y=110, description="Learning Center")
hospital_graph.add_node("Hospital_3F_Sleep_Center", floor=3, x=200, y=120, description="Sleep Center")
hospital_graph.add_node("Hospital_3F_Education", floor=3, x=250, y=130, description="Education Department")
# Add nodes for 4th floor
hospital_graph.add_node("Hospital_4F_Medical_Records", floor=4, x=100, y=100, description="Medical Records")

# Add edges (connections) between rooms/departments
# Connections to first floor (elevator/stairs)
hospital_graph.add_edge("Hospital_1F_Entrance", "Hospital_1F_Reception")
hospital_graph.add_edge("Hospital_1F_Reception", "Hospital_1F_Emergency")
hospital_graph.add_edge("Hospital_1F_Emergency", "Hospital_1F_Womens_Imaging")
hospital_graph.add_edge("Hospital_1F_Womens_Imaging", "Hospital_1F_Diagnostic_Imaging")
hospital_graph.add_edge("Hospital_1F_Diagnostic_Imaging", "Hospital_1F_Laboratory")
hospital_graph.add_edge("Hospital_1F_Laboratory", "Hospital_1F_Courtyard_Cafe")
hospital_graph.add_edge("Hospital_1F_Courtyard_Cafe", "Hospital_1F_Conference_Center")
# Connections to second floor (elevator/stairs)
hospital_graph.add_edge("Hospital_1F_Entrance", "Hospital_2F_Critical_Care")  # Stairs/Elevator
hospital_graph.add_edge("Hospital_2F_Critical_Care", "Hospital_2F_Surgery")
hospital_graph.add_edge("Hospital_2F_Surgery", "Hospital_2F_Outpatient_Infusion")
# Connections to third floor (elevator/stairs)
hospital_graph.add_edge("Hospital_2F_Critical_Care", "Hospital_3F_Human_Resources")  # Elevator/Stairs
hospital_graph.add_edge("Hospital_3F_Human_Resources", "Hospital_3F_Learning_Center")
hospital_graph.add_edge("Hospital_3F_Learning_Center", "Hospital_3F_Sleep_Center")
hospital_graph.add_edge("Hospital_3F_Sleep_Center", "Hospital_3F_Education")
# Connection to fourth floor
hospital_graph.add_edge("Hospital_3F_Learning_Center", "Hospital_4F_Medical_Records")  # Elevator/Stairs

# Function to convert floor number to ordinal string
def floor_to_ordinal(floor):
    ordinal = ["", "1st", "2nd", "3rd", "4th"]
    return ordinal[floor] if floor < len(ordinal) else str(floor) + "th"

# Function to generate navigation instructions and audio
def generate_navigation_with_voice(start_location, end_location):
    start_node = description_to_node[start_location]
    end_node = description_to_node[end_location]

    try:
        shortest_path = nx.shortest_path(hospital_graph, source=start_node, target=end_node)
        navigation_instructions = ""

        for i in range(len(shortest_path) - 1):
            current_location = shortest_path[i]
            next_location = shortest_path[i + 1]
            current_description = hospital_graph.nodes[current_location]['description']
            next_description = hospital_graph.nodes[next_location]['description']
            current_floor = floor_to_ordinal(hospital_graph.nodes[current_location]['floor'])
            next_floor = floor_to_ordinal(hospital_graph.nodes[next_location]['floor'])

            # Add directional instructions based on node coordinates
            if hospital_graph.nodes[current_location]['floor'] == hospital_graph.nodes[next_location]['floor']:
                # Same floor
                navigation_instructions += f"From {current_description} on the {current_floor}, proceed to the right towards {next_description} on the same floor. "
            else:
                # Different floors
                if hospital_graph.nodes[next_location]['floor'] > hospital_graph.nodes[current_location]['floor']:
                    navigation_instructions += f"From {current_description} on the {current_floor}, take the elevator or stairs up to the {next_floor} towards {next_description}. "
                else:
                    navigation_instructions += f"From {current_description} on the {current_floor}, take the elevator or stairs down to the {next_floor} towards {next_description}. "

        # Convert navigation instructions to speech
        tts = gTTS(navigation_instructions, lang='en')
        audio_file = "navigation_instructions.mp3"
        tts.save(audio_file)

        return navigation_instructions, audio_file
    except nx.NetworkXNoPath:
        return "No path found between the specified locations.", None

# Create a mapping of descriptions to node names for the dropdowns
description_to_node = {hospital_graph.nodes[node]['description']: node for node in hospital_graph.nodes}

In [ ]:
# Load pre-trained model and tokenizer for health chatbot
model_name = "microsoft/DialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
# Define health advice and medicines
symptom_data = {
    "headache": {
        "advice": "Rest in a quiet, dark room. Stay hydrated.",
        "medicines": "Ibuprofen, Acetaminophen"
    },
    "fever": {
        "advice": "Stay hydrated, rest, and monitor temperature. Wear light clothing.",
        "medicines": "Acetaminophen, Ibuprofen"
    },
    "cough": {
        "advice": "Stay hydrated, use a humidifier, and rest.",
        "medicines": "Dextromethorphan (cough suppressant), Guaifenesin (expectorant)"
    },
    "sore throat": {
        "advice": "Gargle with salt water, stay hydrated, and rest.",
        "medicines": "Lozenges with menthol, Acetaminophen, Ibuprofen"
    },
    "nausea": {
        "advice": "Eat small, bland meals, and stay hydrated.",
        "medicines": "Dimenhydrinate, Meclizine"
    },
    "vomiting": {
        "advice": "Rehydrate with clear fluids. Avoid solid food until vomiting subsides.",
        "medicines": "Ondansetron, Promethazine"
    },
    "diarrhea": {
        "advice": "Drink oral rehydration solutions, avoid dairy and fatty foods.",
        "medicines": "Loperamide, Bismuth subsalicylate"
    },
    "constipation": {
        "advice": "Increase fiber intake, drink plenty of fluids, and exercise.",
        "medicines": "Psyllium, Polyethylene glycol"
    },
    "abdominal pain": {
        "advice": "Identify and avoid trigger foods. Apply a heating pad.",
        "medicines": "Antacids, Simethicone (for gas), Ibuprofen"
    },
    "back pain": {
        "advice": "Maintain good posture, rest, and apply heat or cold packs.",
        "medicines": "Ibuprofen, Acetaminophen"
    },
    "muscle pain": {
        "advice": "Rest, stretch gently, and apply heat.",
        "medicines": "Ibuprofen, Naproxen"
    },
    "joint pain": {
        "advice": "Rest, use supportive devices if needed, and apply ice.",
        "medicines": "Acetaminophen, Ibuprofen"
    },
    "dizziness": {
        "advice": "Sit or lie down until it passes. Avoid sudden movements.",
        "medicines": "Meclizine, Dimenhydrinate"
    },
    "fatigue": {
        "advice": "Ensure adequate sleep, maintain a balanced diet, and exercise regularly.",
        "medicines": "Consult a doctor; no over-the-counter medicines are specific for fatigue."
    },
    "insomnia": {
        "advice": "Establish a regular sleep routine, avoid caffeine before bed.",
        "medicines": "Melatonin, Diphenhydramine"
    },
    "anxiety": {
        "advice": "Practice relaxation techniques, exercise, and consider counseling.",
        "medicines": "Consult a doctor; medications like Diazepam or Lorazepam might be prescribed."
    },
    "depression": {
        "advice": "Seek counseling, maintain a healthy lifestyle, and consider professional help.",
        "medicines": "Antidepressants like Sertraline, Fluoxetine"
    },
    "skin rash": {
        "advice": "Avoid irritants, apply soothing lotions.",
        "medicines": "Hydrocortisone cream, Antihistamines"
    },
    "allergic reactions": {
        "advice": "Identify and avoid allergens.",
        "medicines": "Antihistamines like Loratadine, Cetirizine"
    },
    "hives": {
        "advice": "Avoid known triggers and use anti-itch lotions.",
        "medicines": "Antihistamines like Diphenhydramine, Hydroxyzine"
    },
    "cold": {
        "advice": "Rest, stay hydrated, and use saline nasal sprays.",
        "medicines": "Decongestants like Pseudoephedrine, Cough drops"
    },
    "sinusitis": {
        "advice": "Use a humidifier, stay hydrated, and apply warm compresses.",
        "medicines": "Decongestants, Nasal corticosteroids"
    },
    "earache": {
        "advice": "Avoid getting water in the ear, and use warm compresses.",
        "medicines": "Acetaminophen, Ibuprofen"
    },
    "eye strain": {
        "advice": "Take regular breaks from screens, adjust lighting.",
        "medicines": "Artificial tears, Antihistamine eye drops if due to allergies."
    },
    "dry mouth": {
        "advice": "Drink water frequently and use sugar-free gum.",
        "medicines": "Saliva substitutes"
    },
    "bad breath": {
        "advice": "Maintain good oral hygiene and stay hydrated.",
        "medicines": "Mouthwashes with antibacterial properties"
    },
    "mouth sores": {
        "advice": "Avoid spicy or acidic foods and maintain oral hygiene.",
        "medicines": "Benzocaine gels, Antiseptic mouth rinses"
    },
    "toothache": {
        "advice": "Avoid very hot or cold foods and maintain oral hygiene.",
        "medicines": "Ibuprofen, Acetaminophen"
    },
    "nasal congestion": {
        "advice": "Use saline nasal sprays and stay hydrated.",
        "medicines": "Decongestants like Oxymetazoline, Phenylephrine"
    },
    "sore muscles": {
        "advice": "Apply heat, stretch, and rest.",
        "medicines": "Ibuprofen, Muscle relaxants if prescribed"
    },
    "swollen feet": {
        "advice": "Elevate feet, reduce salt intake, and exercise.",
        "medicines": "Diuretics (consult a doctor)"
    },
    "swollen hands": {
        "advice": "Elevate hands and reduce salt intake.",
        "medicines": "Diuretics (consult a doctor)"
    },
    "chronic cough": {
        "advice": "See a doctor to determine underlying cause.",
        "medicines": "Depending on the cause, consider medications like cough suppressants or expectorants."
    },
    "shortness of breath": {
        "advice": "Seek immediate medical attention if severe. Use inhalers if prescribed.",
        "medicines": "Bronchodilators, Steroids (as prescribed)"
    },
    "heartburn": {
        "advice": "Avoid spicy and fatty foods, eat smaller meals.",
        "medicines": "Antacids like Tums, H2 blockers like Ranitidine"
    },
    "indigestion": {
        "advice": "Eat smaller meals, avoid trigger foods.",
        "medicines": "Antacids, H2 blockers"
    },
    "bloating": {
        "advice": "Avoid carbonated drinks and foods that cause gas.",
        "medicines": "Simethicone, Activated charcoal"
    },
    "menstrual cramps": {
        "advice": "Use heat pads and exercise lightly.",
        "medicines": "Ibuprofen, Naproxen"
    },
    "pms": {
        "advice": "Exercise, maintain a balanced diet, and manage stress.",
        "medicines": "Ibuprofen, Calcium supplements"
    },
    "acne": {
        "advice": "Keep skin clean, avoid picking at acne.",
        "medicines": "Benzoyl peroxide, Salicylic acid"
    },
    "sunburn": {
        "advice": "Avoid further sun exposure, apply aloe vera gel.",
        "medicines": "Over-the-counter hydrocortisone cream"
    },
    "chapped lips": {
        "advice": "Use lip balm frequently and stay hydrated.",
        "medicines": "Lip balms with beeswax or petroleum jelly"
    },
    "cold sores": {
        "advice": "Avoid touching sores and sharing utensils.",
        "medicines": "Acyclovir cream, Docosanol"
    },
    "warts": {
        "advice": "Avoid touching or scratching warts.",
        "medicines": "Salicylic acid treatments"
    },
    "ingrown toenail": {
        "advice": "Soak feet in warm water and wear comfortable shoes.",
        "medicines": "Topical antibiotics if infected, pain relievers"
    }
}

In [ ]:
# creating a the health care bot
def chat_with_hena(selected_symptoms, history=[]):
    if not selected_symptoms:
        return "No symptoms selected. Please choose at least one symptom.", history, None

    # Sets to store unique advice and medicines
    advice_phrases = set()
    combined_medicines = set()

    # Helper function to split advice into individual phrases
    def split_advice(advice):
        return set(advice.split('. '))

    # Loop through each selected symptom
    for symptom in selected_symptoms:
        if symptom in symptom_data:
            data = symptom_data[symptom]

            # Split the advice into unique phrases and add them to the set
            advice_phrases.update(split_advice(data['advice']))

            # Add medicines to the set to avoid duplicates
            combined_medicines.update(data['medicines'].split(', '))

    # Join the unique advice phrases back into a coherent string
    combined_advice = ". ".join(advice_phrases) + "."
    # Join the unique medicines into a single string
    medicines = ", ".join(combined_medicines)

    # Create the final response
    combined_response = (
        f"For the symptoms: {', '.join(selected_symptoms)}, here's the advice: {combined_advice} "
        f"You can take these medicines: {medicines}. "
        "If you need more help, just let me know!"
    )

    # Append to chat history
    history.append((', '.join(selected_symptoms), combined_response))

    # Convert the response to speech
    tts = gTTS(combined_response, lang='en')
    audio_file = "chatbot_response.mp3"
    tts.save(audio_file)

    return combined_response, history, audio_file

In [ ]:
custom_css = """
    #chatbot {
        background-color: #f0f8ff;
        padding: 20px;
        border-radius: 10px;
        box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
    }

    .gr-button {
        background-color: #4CAF50;
        color: white;
        font-size: 16px;
        padding: 10px 24px;
        border-radius: 8px;
        margin: 10px 0;
    }

    .gr-button:hover {
        background-color: #45a049;
    }

    #navigation-panel {
        background-color:#ffebcd ;
        border-radius: 10px;
        padding: 15px;
        box-shadow: 0px 4px 8px rgba(0, 0, 0, 0.1);
    }

    #symptoms-panel {
        background-color: #ffebcd;
        border-radius: 8px;
        padding: 20px;
        box-shadow: 0px 4px 8px rgba(0, 0, 0, 0.1);
    }

    .gr-dropdown, .gr-checkbox-group, .gr-textbox {
        border-radius: 8px;
        padding: 12px;
        margin-bottom: 15px;
    }

    .gr-audio {
        margin-top: 20px;
    }
"""

# Creating the Gradio app with styling
with gr.Blocks(css=custom_css) as demo:

    with gr.Tab("Hospital Building"):
        gr.Image("/content/GMC-floor1-2023.png", label="1st Floor")
        gr.Image("/content/GMC-floor2-2023.png", label="2nd Floor")
        gr.Image("/content/GMC-floor3-2023.png", label="3rd Floor")
        gr.Image("/content/GMC-floor4-2023.png", label="4th Floor")

    with gr.Tab("Administration Building"):
        gr.Image("/content/MOB1.png", label="1st Floor")
        gr.Image("/content/MOB-2.png", label="2nd Floor")
        gr.Image("/content/MOB-3.png", label="3rd Floor")

    with gr.Tab("Navigation", elem_id="navigation-panel"):
        start_location = gr.Dropdown(choices=list(description_to_node.keys()), label="Start Location", elem_id="dropdown-start")
        end_location = gr.Dropdown(choices=list(description_to_node.keys()), label="End Location", elem_id="dropdown-end")
        navigate_button = gr.Button("Navigate", elem_id="navigate-btn")
        output_text = gr.Textbox(label="Navigation Instructions")
        output_audio = gr.Audio(label="Navigation Audio", type="filepath", autoplay=True, visible=False)

        navigate_button.click(fn=generate_navigation_with_voice,
                              inputs=[start_location, end_location],
                              outputs=[output_text, output_audio])

        # Force play the audio through JavaScript
        gr.HTML("""
        <script>
            const audioElements = document.querySelectorAll('audio');
            window.addEventListener('DOMContentLoaded', () => {
                audioElements.forEach(audio => {
                    audio.muted = false;
                    audio.play();
                });
            });
        </script>
        """)


    with gr.Tab("Health Care", elem_id="symptoms-panel"):
        symptom_choices = list(symptom_data.keys())
        selected_symptoms = gr.CheckboxGroup(choices=symptom_choices, label="Select Your Symptoms", elem_id="symptoms")
        chatbot_output = gr.Textbox(label="Chatbot Response", elem_id="chatbot")
        output_audio_chatbot = gr.Audio(label="Chatbot Audio Response", type="filepath", autoplay=True, visible=False)
        chatbot_state = gr.State()
        chatbot_button = gr.Button("Ask HENA", elem_id="chatbot-btn")

        chatbot_button.click(fn=chat_with_hena,
                             inputs=[selected_symptoms],
                             outputs=[chatbot_output, chatbot_state, output_audio_chatbot])

        # Force play the audio through JavaScript
        gr.HTML("""
        <script>
            const audioElements = document.querySelectorAll('audio');
            window.addEventListener('DOMContentLoaded', () => {
                audioElements.forEach(audio => {
                    audio.muted = false;
                    audio.play();
                });
            });
        </script>
        """)


In [ ]:
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://66115cf2bb8c7537f8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
